Notes:
- Using pandas.cut for binning. bin_min=-inf, bin_max=inf. Binning on normalized data only.* (Other option to explore: sklearn KBinsDiscretizer. Issue is that bins cant be predefined. We need same bins for each batch of y. advantage: inverse is easily available)

- Changing y to categorical only for training. not for valid, test. so shape of y will be different. shape[batch_size, lat, lon, variable, member] Is shape ok?

- Please check custom loss function in networks.py, create_predictions() , train(), build_resnet_categorical()

ToDo
- currently works only for 2 output variables.
- sampling from output to create an ensemble.
- use diff. in temp instead of absolute values.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from src.data_generator import *
from src.train import *
from src.utils import *
from src.networks import *

In [3]:
tf.__version__ #gotta check. some issue. not able to use gpu.

'2.0.0'

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]=str(5)
limit_mem()

In [5]:
args = load_args('../nn_configs/B/81.1-resnet_d3_dr_0.1.yml')

In [6]:
args['train_years']=['2017']
args['valid_years']=['2018']
args['test_years']=['2018']
args['model_save_dir'] ='/home/garg/data/WeatherBench/predictions/saved_models'
args['datadir']='/home/garg/data/WeatherBench/5.625deg'

In [7]:
args['is_categorical']=True
args['num_bins'], args['bin_min'], args['bin_max']
num_bins=args['num_bins']

In [8]:
args['filters'] = [128, 128, 128, 128, 128, 128, 128, 128, 
                   128, 128, 128, 128, 128, 128, 128, 128, 
                   128, 128, 128, 128, 2*num_bins]
args['loss'] = 'lat_categorical_loss'
#could change it directly in build_resnet_categorical fn. Should we?

In [9]:
dg_train, dg_valid, dg_test = load_data(**args)

In [10]:
x,y=dg_train[0]; print(x.shape, y.shape)
x,y=dg_valid[0]; print(x.shape, y.shape) 
x,y=dg_test[0]; print(x.shape, y.shape)
#only changing train shape.

(32, 32, 64, 114) (32, 32, 64, 2, 50)
(32, 32, 64, 114) (32, 32, 64, 2)
(32, 32, 64, 114) (32, 32, 64, 2)


In [11]:
model = build_resnet_categorical(
    **args, input_shape=dg_train.shape,
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 64, 114) 0                                            
__________________________________________________________________________________________________
periodic_conv2d (PeriodicConv2D (None, 32, 64, 128)  715136      input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 32, 64, 128)  0           periodic_conv2d[0][0]            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 64, 128)  512         leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [12]:
args['loss']

'lat_categorical_loss'

In [13]:
model.compile(keras.optimizers.Adam(1e-3), loss=args['loss'])

ValueError: Unknown loss function:lat_categorical_loss

In [14]:
model.compile(keras.optimizers.Adam(1e-3), loss=lat_categorical_loss)

NotImplementedError: Cannot convert a symbolic Tensor (tf_op_layer_stack_target:0) to a numpy array.

In [13]:
def categorical_loss(y_true, y_pred):    
        cce=tf.keras.losses.CategoricalCrossentropy()
        loss=0 #is this ok?
        for i in range(2):
            loss +=cce(y_true[:,:,:,i,:], y_pred[:,:,:,i,:])
        return loss

In [14]:
model.compile(keras.optimizers.Adam(1e-3), loss=categorical_loss)

In [15]:
model.fit(dg_train, epochs=1, shuffle=False)

Train for 136 steps
  3/136 [..............................] - ETA: 35:25 - loss: 21.8042

KeyboardInterrupt: 

In [ ]:
#exp_id=args['exp_id']
exp_id='categorical_v3'
model_save_dir=args['model_save_dir']

model.save(f'{model_save_dir}/{exp_id}.h5')
model.save_weights(f'{model_save_dir}/{exp_id}_weights.h5')
#to_pickle(history.history, f'{model_save_dir}/{exp_id}_history.pkl')

In [16]:
exp_id='categorical_v3'
model = keras.models.load_model(f"{args['model_save_dir']}/{exp_id}.h5", 
                                custom_objects={'PeriodicConv2D': PeriodicConv2D, 'categorical_loss': tf.keras.losses.mse})
#check how to call loss correctly.

In [70]:
#preds = create_predictions(model, dg_valid, is_categorical=True)
#Check in datagenerator and train()

In [73]:
preds=model.predict(dg_valid[0])
print(preds.shape)
preds=preds.reshape(preds.shape[0], preds.shape[1], preds.shape[2], preds.shape[4], preds.shape[3])
print(preds.shape)

(32, 32, 64, 2, 50)
(32, 32, 64, 50, 2)


In [74]:
#JUST for testing create_predictions() on single batch. Ignore this cell block.

dg=dg_valid
level_names = dg.data.isel(level=dg.output_idxs).level_names
level = dg.data.isel(level=dg.output_idxs).level
    
preds = xr.DataArray(
        preds,
        dims=['time', 'lat', 'lon', 'member', 'level'],
        coords={'time': dg.valid_time[0:32], 'lat': dg.data.lat, 'lon': dg.data.lon,
                'level': level,
                'member': np.arange(num_bins),
                'level_names': level_names,
                },
    )  

# Unnormalize
mean = dg.mean.isel(level=dg.output_idxs).values
std = dg.std.isel(level=dg.output_idxs).values
preds = preds * std + mean

unique_vars = list(set([l.split('_')[0] for l in preds.level_names.values]))

das = []
for v in unique_vars:
    idxs = [i for i, vv in enumerate(preds.level_names.values) if vv.split('_')[0] in v]
    da = preds.isel(level=idxs).squeeze().drop('level_names')
    if not 'level' in da.dims: da = da.drop('level')
    das.append({v: da})
preds=xr.merge(das)

In [76]:
preds

<xarray.Dataset>
Dimensions:  (lat: 32, lon: 64, member: 50, time: 32)
Coordinates:
  * time     (time) datetime64[ns] 2017-01-04T04:00:00 ... 2017-01-06T18:00:00
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * member   (member) int64 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49
Data variables:
    t        (time, lat, lon, member) float64 274.0 274.0 274.0 ... 274.0 274.0
    z        (time, lat, lon, member) float64 5.405e+04 5.405e+04 ... 5.405e+04

In [77]:
preds.to_netcdf('/home/garg/data/WeatherBench/predictions/categorical_v3.nc')

In [80]:
args['datadir']

'/home/garg/data/WeatherBench/5.625deg'

In [87]:
datadir=args['datadir']
z500_valid = load_test_data(f'{datadir}/geopotential_500', 'z').drop('level')
t850_valid = load_test_data(f'{datadir}/temperature_850', 't').drop('level')
valid = xr.merge([z500_valid, t850_valid]).sel(time=preds.time)

In [88]:
valid

<xarray.Dataset>
Dimensions:  (lat: 32, lon: 64, time: 32)
Coordinates:
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 2017-01-04T04:00:00 ... 2017-01-06T18:00:00
Data variables:
    z        (time, lat, lon) float32 dask.array<chunksize=(32, 32, 64), meta=np.ndarray>
    t        (time, lat, lon) float32 dask.array<chunksize=(32, 32, 64), meta=np.ndarray>

In [89]:
ensemblemean=preds.mean('member') #ensemble
mean_rmse_p=compute_weighted_rmse(valid,ensemblemean).load(); print(mean_rmse_p)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    z_rmse   float64 2.956e+03
    t_rmse   float64 13.93


In [92]:
crps=compute_weighted_crps(preds,valid).load(); print(crps)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 2.656e+03
    t        float64 12.22
